# منصة تحليل العقارات في الرياض
# Riyadh Real Estate Analytics Platform

هذا دفتر ملاحظات تفاعلي يوضح قدرات منصة التحليل العقاري لمدينة الرياض

This notebook demonstrates the capabilities of the Riyadh Real Estate Analytics Platform.

## 1. إعداد البيئة | Environment Setup

In [ ]:
import sys
sys.path.append('..')

from riyadh_real_estate import RiyadhRealEstateData, PricePredictor, MarketAnalyzer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ تم تحميل جميع المكتبات بنجاح")
print("✓ All libraries loaded successfully")

## 2. تحميل ومعالجة البيانات | Data Loading and Processing

In [ ]:
# Initialize data processor
data_processor = RiyadhRealEstateData()

# Create sample data for demonstration
# في التطبيق الفعلي، استخدم: data_processor.load_data('path/to/data.csv')
df = data_processor.create_sample_data(n_samples=500)

print(f"عدد العقارات: {len(df)}")
print(f"Number of properties: {len(df)}\n")

# Display first few rows
df.head()

## 3. إحصائيات البيانات | Data Statistics

In [ ]:
# Get statistics
stats = data_processor.get_statistics()

print("الإحصائيات الأساسية | Basic Statistics:")
print("=" * 50)
for key, value in stats.items():
    if value is not None:
        if isinstance(value, float):
            print(f"{key}: {value:,.2f}")
        else:
            print(f"{key}: {value}")

In [ ]:
# Data distribution
df.describe()

## 4. تحليل السوق | Market Analysis

In [ ]:
# Initialize market analyzer
analyzer = MarketAnalyzer(df)

# Get market trends
trends = analyzer.get_market_trends()

print("اتجاهات السوق | Market Trends:")
print("=" * 50)
for key, value in trends.items():
    print(f"{key}: {value}")

In [ ]:
# Analysis by district
district_analysis = analyzer.analyze_by_district()
print("التحليل حسب الحي | Analysis by District:")
district_analysis

In [ ]:
# Analysis by property type
type_analysis = analyzer.analyze_by_property_type()
print("التحليل حسب نوع العقار | Analysis by Property Type:")
type_analysis

## 5. التصورات البيانية | Data Visualizations

In [ ]:
# Price distribution
plt.figure(figsize=(12, 6))
plt.hist(df['السعر_ريال'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
plt.xlabel('السعر (ريال سعودي)', fontsize=12)
plt.ylabel('عدد العقارات', fontsize=12)
plt.title('توزيع أسعار العقارات في الرياض', fontsize=14, pad=20)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Average price by district
district_avg = df.groupby('الحي')['السعر_ريال'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
district_avg.plot(kind='bar', color='coral', edgecolor='black')
plt.xlabel('الحي', fontsize=12)
plt.ylabel('متوسط السعر (ريال سعودي)', fontsize=12)
plt.title('متوسط أسعار العقارات حسب الحي', fontsize=14, pad=20)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Price by property type
plt.figure(figsize=(10, 6))
df.boxplot(column='السعر_ريال', by='نوع_العقار', figsize=(10, 6))
plt.xlabel('نوع العقار', fontsize=12)
plt.ylabel('السعر (ريال سعودي)', fontsize=12)
plt.title('توزيع الأسعار حسب نوع العقار', fontsize=14)
plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

In [ ]:
# Correlation heatmap
numerical_cols = df.select_dtypes(include=[np.number]).columns

plt.figure(figsize=(10, 8))
sns.heatmap(
    df[numerical_cols].corr(),
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=1,
    cbar_kws={'label': 'معامل الارتباط'}
)
plt.title('مصفوفة الارتباط بين المتغيرات', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

## 6. بناء نموذج التنبؤ | Building Prediction Model

In [ ]:
# Clean data and prepare features
cleaned_data = data_processor.clean_data()
X, y = data_processor.get_feature_matrix()

print(f"عدد الميزات: {X.shape[1]}")
print(f"Number of features: {X.shape[1]}")
print(f"\nأسماء الميزات | Feature names:")
print(X.columns.tolist())

In [ ]:
# Train prediction model
predictor = PricePredictor(model_type='random_forest')
metrics = predictor.train(X, y, test_size=0.2)

print("مقاييس أداء النموذج | Model Performance Metrics:")
print("=" * 50)
print(f"التدريب - متوسط الخطأ المطلق: {metrics['train_mae']:,.2f}")
print(f"Training - Mean Absolute Error: {metrics['train_mae']:,.2f}")
print(f"\nالاختبار - متوسط الخطأ المطلق: {metrics['test_mae']:,.2f}")
print(f"Testing - Mean Absolute Error: {metrics['test_mae']:,.2f}")
print(f"\nمعامل التحديد R² للاختبار: {metrics['test_r2']:.4f}")
print(f"Testing R² Score: {metrics['test_r2']:.4f}")

In [ ]:
# Feature importance
importance = predictor.get_feature_importance()
print("أهمية الميزات | Feature Importance:")
print(importance.head(10))

# Plot feature importance
plt.figure(figsize=(10, 6))
top_features = importance.head(10)
plt.barh(range(len(top_features)), top_features['importance'])
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('الأهمية | Importance', fontsize=12)
plt.title('أهم 10 ميزات للتنبؤ بالسعر', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

## 7. التنبؤ بالأسعار | Price Prediction

In [ ]:
# Make predictions on test set
predictions = predictor.predict(X.head(10))
actual = y.head(10).values

comparison_df = pd.DataFrame({
    'السعر_الفعلي': actual,
    'السعر_المتوقع': predictions,
    'الفرق': actual - predictions,
    'نسبة_الخطأ_%': ((actual - predictions) / actual * 100).round(2)
})

print("مقارنة الأسعار الفعلية والمتوقعة | Actual vs Predicted Prices:")
comparison_df

In [ ]:
# Visualize predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(actual, predictions, alpha=0.6, s=100)
plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'r--', lw=2)
plt.xlabel('السعر الفعلي (ريال)', fontsize=12)
plt.ylabel('السعر المتوقع (ريال)', fontsize=12)
plt.title('مقارنة الأسعار الفعلية والمتوقعة', fontsize=14, pad=20)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 8. أمثلة على التنبؤ | Prediction Examples

In [ ]:
# Example: Predict price for a specific property
print("مثال 1: شقة في العليا | Example 1: Apartment in Al Olaya")
print("=" * 60)

# Note: This is a simplified example. In practice, you'd need to create
# a feature vector matching the model's expected features
sample_property = X.iloc[0].to_dict()
predicted_price = predictor.predict_single(sample_property)

print(f"السعر المتوقع: {predicted_price:,.2f} ريال سعودي")
print(f"Predicted Price: {predicted_price:,.2f} SAR")

## 9. العقارات ذات القيمة الأفضل | Best Value Properties

In [ ]:
# Find best value properties
best_value = analyzer.find_best_value_properties(n=10)
print("أفضل 10 عقارات من حيث القيمة (أقل سعر للمتر المربع)")
print("Top 10 Best Value Properties (Lowest Price per SQM)")
print("=" * 60)
best_value[['الحي', 'نوع_العقار', 'المساحة_متر_مربع', 'السعر_ريال', 'السعر_للمتر_المربع']].head(10)

## 10. الخلاصة | Summary

### القدرات الرئيسية للمنصة | Main Platform Capabilities:

1. **معالجة البيانات** - تحميل وتنظيف بيانات العقارات
   - Data Processing - Loading and cleaning real estate data

2. **التحليل السوقي** - تحليل السوق حسب الحي ونوع العقار
   - Market Analysis - Analysis by district and property type

3. **التنبؤ بالأسعار** - استخدام التعلم الآلي للتنبؤ بأسعار العقارات
   - Price Prediction - Using machine learning to predict property prices

4. **التصورات البيانية** - رسوم بيانية شاملة لفهم السوق
   - Visualizations - Comprehensive charts for market understanding

5. **تحديد القيمة** - إيجاد أفضل العقارات من حيث القيمة
   - Value Identification - Finding best value properties